We will generate a number of count tables, described in sections below. 

## import 

In [23]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql import functions as F
import pyspark.sql.types as T
import pickle
import pandas as pd

## create a spark session

In [2]:
conf = SparkConf() \
    .setAppName("Count")\

# Create a SparkContext with the specified configurations
if 'spark' in locals() and spark!=None:
    spark.stop()

sc = SparkContext(conf=conf)

# Create a SparkSession from the SparkContext
spark = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/19 17:43:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Load in gnomad variants annotated in the last script

In [3]:
df = spark.read \
    .option("comment", "#") \
    .option("delimiter", ",") \
    .csv("/gpfs/gibbs/pi/reilly/VariantEffects/scripts/noon_data/1.annotate/batched/*.csv/*.csv", header=True)

24/01/19 17:43:38 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## cast columns to the appropriate types & Drop columns rows with null values. 

Dropping isn't strictly necessary. We could, for example, only drop those rows with null malinouis skew when computing malinouis-skew-based metrics, drop rows with no phyloP scores when computing phyloP-based metrics, etc etc. However, this would result in different sets of variants summarized by each graph, which could create biases : if, for example, PhyloP scores are annotated for a nonrandom set of variants. Therefore I will drop rows with null data in any relevant columns prior to subsequent analysis. 

In [4]:
int_columns=["POS","AC","AN"]
float_columns=["AF","K562__ref","HepG2__ref","SKNSH__ref","K562__alt","HepG2__alt","SKNSH__alt","K562__skew","HepG2__skew","SKNSH__skew","cadd_phred","P_ANNO","mean_ref","mean_skew","MAF"]
cre_bool_columns=[]
for column in df.columns:
    if column.startswith("is_in"):
        cre_bool_columns.append(column)

In [5]:
df = df.dropna()#subset=["CHROM","POS","cadd_phred","P_ANNO","mean_ref","mean_skew","category"]+cre_bool_columns

In [6]:

for column in int_columns:
    df = df.withColumn(column, F.col(column).cast(T.IntegerType()))

for column in float_columns:
    df = df.withColumn(column, F.col(column).cast(T.FloatType()))

for column in cre_bool_columns:
    df = df.withColumn(column, F.col(column).cast(T.BooleanType()))

df_cre=df

### Compute pleitropy

"Pleitropy" here refers to a variant which is an emVar in multiple cell-types. We're calling emVars as anything with abs(skew)>=0.5 and max(alt activitym ref activity)>=1

In [26]:
#first we compute whether each variant can be called an emvar in each cell-type. 
for cell_type in ["K562","SKNSH","HepG2"]:
    df_cre = df_cre.withColumn(f"emVar_{cell_type}", 
                           (F.abs(F.col(f"{cell_type}__skew")) >= 0.5) & 
                           (F.greatest(F.col(f"{cell_type}__ref"), F.col(f"{cell_type}__alt")) >= 1.0))

#next, we count the number of cell-types each variant is an emvar in to compute the pleitropy. 
df_cre = df_cre.withColumn("pleio", F.col("emVar_K562").cast("int") + F.col("emVar_SKNSH").cast("int") + F.col("emVar_HepG2").cast("int"))

In [28]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_cre.filter)
    #vcf = vcf.filter((F.col("CHROM") == os.environ['which_chr']))
    display(df_cre.filter(F.col("pleio")>0).limit(30).toPandas())

<bound method DataFrame.filter of DataFrame[CHROM: string, POS: int, ID: string, REF: string, ALT: string, QUAL: string, FILTER: string, INFO: string, K562__ref: float, HepG2__ref: float, SKNSH__ref: float, K562__alt: float, HepG2__alt: float, SKNSH__alt: float, K562__skew: float, HepG2__skew: float, SKNSH__skew: float, AC: int, AN: int, AF: float, cadd_phred: float, is_in_dELS: boolean, is_in_CA: boolean, is_in_pELS: boolean, is_in_CA-H3K4me3: boolean, is_in_CA-CTCF: boolean, is_in_PLS: boolean, is_in_TF: boolean, is_in_CA-TF: boolean, P_ANNO: float, mean_ref: float, mean_skew: float, MAF: float, category: string, emVar_K562: boolean, emVar_SKNSH: boolean, emVar_HepG2: boolean, pleio: int]>

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,K562__ref,HepG2__ref,SKNSH__ref,K562__alt,HepG2__alt,SKNSH__alt,K562__skew,HepG2__skew,SKNSH__skew,AC,AN,AF,cadd_phred,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,is_in_PLS,is_in_TF,is_in_CA-TF,P_ANNO,mean_ref,mean_skew,MAF,category,emVar_K562,emVar_SKNSH,emVar_HepG2,pleio
0,chr1,97908,rs1277319205,T,C,.,PASS,K562__ref=2.9331021;HepG2__ref=2.3081238;SKNSH...,2.933102,2.308124,2.340445,3.849305,3.179416,3.842669,0.916203,0.871293,1.502224,4,121472,0.000033,5.701000,False,False,False,False,False,False,False,False,-0.721,2.527224,1.096573,0.000033,ULTRARARE,True,True,True,3
1,chr1,826965,rs74045218,A,G,.,PASS,K562__ref=1.0931134;HepG2__ref=1.6486894;SKNSH...,1.093113,1.648689,1.751931,1.589439,2.261082,2.285886,0.496325,0.612393,0.533955,8289,148320,0.055886,5.965000,False,False,False,False,False,True,False,False,-0.511,1.497911,0.547558,0.055886,COMMON,False,True,True,2
2,chr1,864215,rs151181074,A,G,.,PASS,K562__ref=0.09337018;HepG2__ref=0.19248968;SKN...,0.093370,0.192490,0.116100,0.662385,1.350148,0.958279,0.569014,1.157658,0.842179,111,152224,0.000729,1.618000,False,False,False,False,False,False,False,False,-3.415,0.133987,0.856284,0.000729,RARE,False,False,True,1
3,chr1,949989,rs906018631,G,A,.,PASS,K562__ref=1.2441471;HepG2__ref=0.55029184;SKNS...,1.244147,0.550292,-0.143183,0.297883,0.146642,-0.369154,-0.946264,-0.403650,-0.225972,1,152148,0.000007,0.031000,False,False,False,False,False,False,False,False,0.746,0.550419,-0.525295,0.000007,SINGLETON,True,False,False,1
4,chr1,954156,rs761614862,G,A,.,PASS,K562__ref=0.097240366;HepG2__ref=0.30659825;SK...,0.097240,0.306598,0.300613,0.177758,1.502077,1.806074,0.080517,1.195479,1.505461,4,152186,0.000026,0.677000,True,False,False,False,False,False,False,False,-0.933,0.234817,0.927153,0.000026,ULTRARARE,False,True,True,2
5,chr1,956761,rs187097013,C,T,.,PASS,K562__ref=1.1145127;HepG2__ref=0.9473483;SKNSH...,1.114513,0.947348,0.190719,0.556268,0.465254,-0.363956,-0.558245,-0.482094,-0.554676,236,152232,0.001550,0.023000,True,False,False,False,False,False,False,False,-2.898,0.750860,-0.531672,0.001550,LOW_FREQ,True,False,False,1
6,chr1,964101,rs369448294,C,T,.,PASS,K562__ref=0.8554277;HepG2__ref=1.4298428;SKNSH...,0.855428,1.429843,0.508925,0.313335,0.451565,-0.107680,-0.542093,-0.978278,-0.616606,5,152184,0.000033,5.716000,False,False,False,False,False,False,False,False,-2.215,0.931399,-0.712325,0.000033,ULTRARARE,False,False,True,1
7,chr1,1009374,rs1026835560,C,A,.,PASS,K562__ref=0.7309728;HepG2__ref=1.1401274;SKNSH...,0.730973,1.140127,1.966433,0.436032,0.597042,1.316521,-0.294941,-0.543086,-0.649912,1,152036,0.000007,0.531000,False,False,True,False,False,False,False,False,0.305,1.279178,-0.495980,0.000007,SINGLETON,False,True,True,2
8,chr1,1009374,rs1026835560,C,G,.,PASS,K562__ref=0.7309728;HepG2__ref=1.1401274;SKNSH...,0.730973,1.140127,1.966433,0.369768,0.474611,0.914330,-0.361204,-0.665516,-1.052104,35,152036,0.000230,0.578000,False,False,True,False,False,False,False,False,0.305,1.279178,-0.692941,0.000230,RARE,False,True,True,2
9,chr1,1014451,rs116002608,C,T,.,PASS,K562__ref=3.5883005;HepG2__ref=3.9088778;SKNSH...,3.588300,3.908878,4.636027,2.709045,3.545144,4.306787,-0.879256,-0.363734,-0.329240,1713,152160,0.011258,1.703000,False,False,True,False,False,False,False,False,-2.054,4.044402,-0.524077,0.011258,COMMON,True,False,False,1


# compute count tables

All count tables will be broken down by each of the CRE types. 

## PhyloP vs rarity
- add column : "significant"/"not significant" : threshold is 2.27
- count table of significance VS category
- dump to disc

In [10]:
df_phylop_significant=df_cre.withColumn("phylop_significant",F.col("P_ANNO")>=2.27)

phylop_count_table = df_phylop_significant.groupBy(["category","phylop_significant"]+cre_bool_columns).count()

data_base_path="/home/mcn26/varef/scripts/noon_data/2.count/"

phylop_count_table.coalesce(1).write.csv(data_base_path+"phylop_count_table", mode="overwrite", header=True)

ERROR:root:KeyboardInterrupt while sending command.             (0 + 0) / 12990]
Traceback (most recent call last):
  File "/home/mcn26/.conda/envs/mcn_vareff/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/mcn26/.conda/envs/mcn_vareff/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/mcn26/.conda/envs/mcn_vareff/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
phylop_count_table.toPandas()

## PhyloP VS pleiotropy

## CADD vs rarity
Similar approach to phylop above,

Cutoffs are 
- All
- score≥10
- score≥20
- score≥30
- score≥40
- score≥50

In [34]:
df_cadd_cutoff=df_cre.withColumn(
    "CADD>=10",F.col("cadd_phred")>=10
).withColumn(
    "CADD>=20",F.col("cadd_phred")>=20
).withColumn(
    "CADD>=30",F.col("cadd_phred")>=30
).withColumn(
    "CADD>=40",F.col("cadd_phred")>=40
).withColumn(
    "CADD>=50",F.col("cadd_phred")>=50
)

cadd_columns=["CADD>=10","CADD>=20","CADD>=30","CADD>=40","CADD>=50"]

with open("cadd_columns.pkl",'wb') as file:
    pickle.dump(cadd_columns,file)

cadd_count_table = df_cadd_cutoff.groupBy(["category"]+cadd_columns+cre_bool_columns).count()

data_base_path="/home/mcn26/varef/scripts/noon_data/2.count/"

cadd_count_table.coalesce(1).write.csv(data_base_path+"CADD_count_table", mode="overwrite", header=True)

## malinouis : reference activity & skew vs rarity

First, compute min and max of skew, reference activity.

While this does require aggregation of the entire dataset, min & max specifically ought to be fairly inexpensive to compute.

This will only be performed during initial testing, then values will be recorded & made into constants. 

In [35]:
## add a mean_alt column
df_cre=df_cre.withColumn("mean_alt", (F.col("K562__alt") + F.col("HepG2__alt") + F.col("SKNSH__alt")) / 3)

### helper functions

In [36]:
def get_column_names(var):
    final_names=[]
    for sub in var:
        final_names.append(sub[0])
    return final_names

def dump_cutoff_names_to_disc(var,name):
    #so we don't have to hard-code the names in multiple files. 
    with open(name+'.pkl', 'wb') as file:
        final_names=get_column_names(var)
        pickle.dump(final_names, file)

#(-Inf,1), [1,2), [2,4), [4,6), [6,Inf) (note first bin we would call as not active)
def make_cutoff(name):
    return [
        [name+"_(-Inf,1)",(F.col(name)<1)],
        [name+"_[1,2)",(F.col(name)>=1)&(F.col(name)<2)],
        [name+"_[2,3)",(F.col(name)>=2)&(F.col(name)<3)],
        [name+"_[3,4)",(F.col(name)>=3)&(F.col(name)<4)],
        [name+"_[4,5)",(F.col(name)>=4)&(F.col(name)<5)],
        [name+"_[5,6)",(F.col(name)>=5)&(F.col(name)<6)],
        [name+"_[6,Inf)",(F.col(name)>=6)]
    ]

def apply_cutoffs(df,cutoffs):
    df_working=df
    for name,cutoff_condition in cutoffs:
        df_working=df_working.withColumn(name,cutoff_condition)
    return df_working

In [37]:
skew_cutoffs = []
start_int = -8  # Start with -1.6 (represented as -8 * 0.2)
end_int = 8     # End with 1.6 (represented as 8 * 0.2)
step_int = 1    # Step of 0.2 (represented as 1 * 0.2)

for i in range(start_int, end_int + step_int, step_int):
    start = i * 0.2
    next_start = (i + step_int) * 0.2

    if start <= -1.4:
        label = "mean_skew_(-Inf, -1.4)"
        condition = (F.col("mean_skew") < -1.4)
    elif start >= 1.4:
        label = "mean_skew_(1.4, Inf)"
        condition = (F.col("mean_skew") >= 1.4)
    else:
        label = f"mean_skew_[{start:.2f}, {next_start:.2f})"
        condition = (F.col("mean_skew") >= start) & (F.col("mean_skew") < next_start)

    skew_cutoffs.append([label, condition])


In [38]:
for i in skew_cutoffs:
    print(i)

['mean_skew_(-Inf, -1.4)', Column<'(mean_skew < -1.4)'>]
['mean_skew_(-Inf, -1.4)', Column<'(mean_skew < -1.4)'>]
['mean_skew_[-1.20, -1.00)', Column<'((mean_skew >= -1.2000000000000002) AND (mean_skew < -1.0))'>]
['mean_skew_[-1.00, -0.80)', Column<'((mean_skew >= -1.0) AND (mean_skew < -0.8))'>]
['mean_skew_[-0.80, -0.60)', Column<'((mean_skew >= -0.8) AND (mean_skew < -0.6000000000000001))'>]
['mean_skew_[-0.60, -0.40)', Column<'((mean_skew >= -0.6000000000000001) AND (mean_skew < -0.4))'>]
['mean_skew_[-0.40, -0.20)', Column<'((mean_skew >= -0.4) AND (mean_skew < -0.2))'>]
['mean_skew_[-0.20, 0.00)', Column<'((mean_skew >= -0.2) AND (mean_skew < 0.0))'>]
['mean_skew_[0.00, 0.20)', Column<'((mean_skew >= 0.0) AND (mean_skew < 0.2))'>]
['mean_skew_[0.20, 0.40)', Column<'((mean_skew >= 0.2) AND (mean_skew < 0.4))'>]
['mean_skew_[0.40, 0.60)', Column<'((mean_skew >= 0.4) AND (mean_skew < 0.6000000000000001))'>]
['mean_skew_[0.60, 0.80)', Column<'((mean_skew >= 0.6000000000000001) AND (

In [39]:
mean_ref_cutoffs=make_cutoff("mean_ref")
dump_cutoff_names_to_disc(mean_ref_cutoffs,"mean_ref_cutoffs")

mean_alt_cutoffs=make_cutoff("mean_alt")
dump_cutoff_names_to_disc(mean_alt_cutoffs,"mean_alt_cutoffs")

In [14]:
##df_cre = df_cre.withColumn("mean_skew", F.round(df["mean_skew"], 2))

In [40]:
df_cre=apply_cutoffs(df_cre,skew_cutoffs)
df_cre=apply_cutoffs(df_cre,mean_alt_cutoffs)
df_cre=apply_cutoffs(df_cre,mean_ref_cutoffs)

In [41]:
df_cre_backup=df_cre

In [42]:
df_cre

DataFrame[CHROM: string, POS: int, ID: string, REF: string, ALT: string, QUAL: string, FILTER: string, INFO: string, K562__ref: float, HepG2__ref: float, SKNSH__ref: float, K562__alt: float, HepG2__alt: float, SKNSH__alt: float, K562__skew: float, HepG2__skew: float, SKNSH__skew: float, AC: int, AN: int, AF: float, cadd_phred: float, is_in_dELS: boolean, is_in_CA: boolean, is_in_pELS: boolean, is_in_CA-H3K4me3: boolean, is_in_CA-CTCF: boolean, is_in_PLS: boolean, is_in_TF: boolean, is_in_CA-TF: boolean, P_ANNO: float, mean_ref: float, mean_skew: float, MAF: float, category: string, mean_alt: double, mean_skew_(-Inf, -1.4): boolean, mean_skew_[-1.20, -1.00): boolean, mean_skew_[-1.00, -0.80): boolean, mean_skew_[-0.80, -0.60): boolean, mean_skew_[-0.60, -0.40): boolean, mean_skew_[-0.40, -0.20): boolean, mean_skew_[-0.20, 0.00): boolean, mean_skew_[0.00, 0.20): boolean, mean_skew_[0.20, 0.40): boolean, mean_skew_[0.40, 0.60): boolean, mean_skew_[0.60, 0.80): boolean, mean_skew_[0.80, 1.

In [43]:
to_group_by=["category"]+cre_bool_columns+get_column_names(skew_cutoffs)+get_column_names(mean_ref_cutoffs)+get_column_names(mean_alt_cutoffs)

In [44]:
#some of the column names have commas, which can cause a problem. Let's replace them with underscores.
renamed_column_map = {col: col.replace(',', '^').replace('.','&') for col in to_group_by}

for old_name, new_name in renamed_column_map.items():
    df_cre = df_cre.withColumnRenamed(old_name, new_name)

In [45]:
skew_and_activity_table = df_cre.groupBy(list(renamed_column_map.values())).count()

In [46]:
skew_and_activity_table

DataFrame[category: string, is_in_dELS: boolean, is_in_CA: boolean, is_in_pELS: boolean, is_in_CA-H3K4me3: boolean, is_in_CA-CTCF: boolean, is_in_PLS: boolean, is_in_TF: boolean, is_in_CA-TF: boolean, mean_skew_(-Inf^ -1&4): boolean, mean_skew_[-1&20^ -1&00): boolean, mean_skew_[-1&00^ -0&80): boolean, mean_skew_[-0&80^ -0&60): boolean, mean_skew_[-0&60^ -0&40): boolean, mean_skew_[-0&40^ -0&20): boolean, mean_skew_[-0&20^ 0&00): boolean, mean_skew_[0&00^ 0&20): boolean, mean_skew_[0&20^ 0&40): boolean, mean_skew_[0&40^ 0&60): boolean, mean_skew_[0&60^ 0&80): boolean, mean_skew_[0&80^ 1&00): boolean, mean_skew_[1&00^ 1&20): boolean, mean_skew_[1&20^ 1&40): boolean, mean_skew_(1&4^ Inf): boolean, mean_ref_(-Inf^1): boolean, mean_ref_[1^2): boolean, mean_ref_[2^3): boolean, mean_ref_[3^4): boolean, mean_ref_[4^5): boolean, mean_ref_[5^6): boolean, mean_ref_[6^Inf): boolean, mean_alt_(-Inf^1): boolean, mean_alt_[1^2): boolean, mean_alt_[2^3): boolean, mean_alt_[3^4): boolean, mean_alt_[4^

In [47]:
skew_and_activity_table.coalesce(1).write.csv(data_base_path+"malinouis_skew_and_thresh", mode="overwrite", header=True)

In [48]:
from pyspark.sql.functions import col
from functools import reduce
from operator import or_

condition = reduce(or_, [col(c).isNull() for c in df_cre.columns])

# Applying the filter
null_rows = df_cre.filter(condition)
z=null_rows.toPandas()

#df_cre["mean_skew_[-1&5^ -1)"]

In [49]:
import pandas as pd
with pd.option_context('display.max_columns', None):
    display(z)


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,K562__ref,HepG2__ref,SKNSH__ref,K562__alt,HepG2__alt,SKNSH__alt,K562__skew,HepG2__skew,SKNSH__skew,AC,AN,AF,cadd_phred,is_in_dELS,is_in_CA,is_in_pELS,is_in_CA-H3K4me3,is_in_CA-CTCF,is_in_PLS,is_in_TF,is_in_CA-TF,P_ANNO,mean_ref,mean_skew,MAF,category,mean_alt,mean_skew_(-Inf^ -1&4),mean_skew_[-1&20^ -1&00),mean_skew_[-1&00^ -0&80),mean_skew_[-0&80^ -0&60),mean_skew_[-0&60^ -0&40),mean_skew_[-0&40^ -0&20),mean_skew_[-0&20^ 0&00),mean_skew_[0&00^ 0&20),mean_skew_[0&20^ 0&40),mean_skew_[0&40^ 0&60),mean_skew_[0&60^ 0&80),mean_skew_[0&80^ 1&00),mean_skew_[1&00^ 1&20),mean_skew_[1&20^ 1&40),mean_skew_(1&4^ Inf),mean_alt_(-Inf^1),mean_alt_[1^2),mean_alt_[2^3),mean_alt_[3^4),mean_alt_[4^5),mean_alt_[5^6),mean_alt_[6^Inf),mean_ref_(-Inf^1),mean_ref_[1^2),mean_ref_[2^3),mean_ref_[3^4),mean_ref_[4^5),mean_ref_[5^6),mean_ref_[6^Inf)
